In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# import hvplot.pandas
# import holoviews as hv
# import panel as pn
import pandas as pd
import numpy as np
# import param as pm
import random
import math
# pn.extension(sizing_mode="stretch_width")
# hv.extension("bokeh")
import os, sys
sys.path.append('/home/bk/LTF-base/aaa-tokenomics')
# pn.extension('plotly')

In [3]:
from AAA.sim_handler import SimHandler

In [4]:
s = SimHandler()

In [5]:
s.run()

user 1 successfully staked 50 AHM 
user 2 successfully staked 100 AHM 
user 3 successfully staked 100 AHM 
user 4 successfully staked 200 AHM 
[User-1: Bal={'DAI': 1000, 'AHM': 150, 'sAHM': 50.5, 'ETH': 1}, User-2: Bal={'DAI': 1000, 'AHM': 100, 'sAHM': 101.0, 'ETH': 1}, User-3: Bal={'DAI': 1000, 'AHM': 100, 'sAHM': 101.0, 'ETH': 1}, User-4: Bal={'DAI': 1000, 'AHM': 0, 'sAHM': 202.0, 'ETH': 1}]
[User-1: Bal={'DAI': 1000, 'AHM': 150, 'sAHM': 51.005, 'ETH': 1}, User-2: Bal={'DAI': 1000, 'AHM': 100, 'sAHM': 102.01, 'ETH': 1}, User-3: Bal={'DAI': 1000, 'AHM': 100, 'sAHM': 102.01, 'ETH': 1}, User-4: Bal={'DAI': 1000, 'AHM': 0, 'sAHM': 204.02, 'ETH': 1}]
[User-1: Bal={'DAI': 1000, 'AHM': 150, 'sAHM': 51.51505, 'ETH': 1}, User-2: Bal={'DAI': 1000, 'AHM': 100, 'sAHM': 103.0301, 'ETH': 1}, User-3: Bal={'DAI': 1000, 'AHM': 100, 'sAHM': 103.0301, 'ETH': 1}, User-4: Bal={'DAI': 1000, 'AHM': 0, 'sAHM': 206.0602, 'ETH': 1}]
[User-1: Bal={'DAI': 1000, 'AHM': 150, 'sAHM': 52.0302005, 'ETH': 1}, User-2:

In [6]:
s.users

[User-1: Bal={'DAI': 1000, 'AHM': 150, 'sAHM': 55.23110627056022, 'ETH': 1},
 User-2: Bal={'DAI': 1000, 'AHM': 100, 'sAHM': 110.46221254112044, 'ETH': 1},
 User-3: Bal={'DAI': 1000, 'AHM': 100, 'sAHM': 110.46221254112044, 'ETH': 1},
 User-4: Bal={'DAI': 1000, 'AHM': 0, 'sAHM': 220.92442508224087, 'ETH': 1}]